In [4]:
import sys
sys.path.append('C:\\Users\\Yuya\\Repositories\\Python\\escimg')

In [5]:
import esccol
from esccol import EsccolDataSet,EsccolRecord,RGBEsccolRecord
import click
import matplotlib.pyplot as plt
from colorsys import rgb_to_hsv
import csv,tqdm,pickle
from itertools import product
import numpy as np
from sklearn import svm
from sklearn.preprocessing import PolynomialFeatures

コンソール画面の背景は白くすること


In [6]:
import tensorflow as tf
from tensorflow import keras

## 学習データを作る

In [18]:
ds = EsccolDataSet()

In [19]:
with open('esccol_traindata.csv','r')as file:
    reader = csv.reader(file)
    for i,row in enumerate(reader):
        if i==0:continue
        r,g,b,rgb_hex,esccol_number = row
        record = RGBEsccolRecord(r=float(r),g=float(g),b=float(b),esccol_number=int(esccol_number))
        ds.append(record)

In [20]:
ds.records

[RGBEsccolRecord(r=0.0, g=0.0, b=0.0, esccol_number=0),
 RGBEsccolRecord(r=0.0, g=0.0, b=0.1111111111, esccol_number=6),
 RGBEsccolRecord(r=0.0, g=0.0, b=0.2222222222, esccol_number=6),
 RGBEsccolRecord(r=0.0, g=0.0, b=0.3333333333, esccol_number=6),
 RGBEsccolRecord(r=0.0, g=0.0, b=0.4444444444, esccol_number=12),
 RGBEsccolRecord(r=0.0, g=0.0, b=0.5555555556, esccol_number=12),
 RGBEsccolRecord(r=0.0, g=0.0, b=0.6666666667, esccol_number=12),
 RGBEsccolRecord(r=0.0, g=0.0, b=0.7777777778, esccol_number=12),
 RGBEsccolRecord(r=0.0, g=0.0, b=0.8888888889, esccol_number=12),
 RGBEsccolRecord(r=0.0, g=0.0, b=1.0, esccol_number=12),
 RGBEsccolRecord(r=0.0, g=0.1111111111, b=0.0, esccol_number=0),
 RGBEsccolRecord(r=0.0, g=0.1111111111, b=0.1111111111, esccol_number=6),
 RGBEsccolRecord(r=0.0, g=0.1111111111, b=0.2222222222, esccol_number=6),
 RGBEsccolRecord(r=0.0, g=0.1111111111, b=0.3333333333, esccol_number=6),
 RGBEsccolRecord(r=0.0, g=0.1111111111, b=0.4444444444, esccol_number=12),


## NNで学習

In [21]:
N = len(esccol.ESCCODES)
N

15

In [31]:
model = keras.Sequential([
    keras.layers.Dense(32,input_shape=(3,),activation='swish'),
    keras.layers.Dense(64,activation='swish'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dense(N,activation='softmax')
])
model.compile(
    metrics=['accuracy'],
    loss='sparse_categorical_crossentropy',
)

In [32]:
x = []
y = []
for record in ds.records:
    x.append((record.r,record.g,record.b))
    y.append(record.esccol_number)
x = tf.convert_to_tensor(x)
y = tf.convert_to_tensor(y)

In [35]:
history = model.fit(x,y,epochs=30,batch_size=3)

Epoch 1/30
667/667 [==============================] - 0s 593us/step - loss: 0.4170 - accuracy: 0.8485
Epoch 2/30
667/667 [==============================] - 0s 564us/step - loss: 0.3919 - accuracy: 0.8605
Epoch 3/30
667/667 [==============================] - 0s 554us/step - loss: 0.4195 - accuracy: 0.8480
Epoch 4/30
667/667 [==============================] - 0s 554us/step - loss: 0.4031 - accuracy: 0.8520
Epoch 5/30
667/667 [==============================] - 0s 556us/step - loss: 0.4219 - accuracy: 0.8570
Epoch 6/30
667/667 [==============================] - 0s 550us/step - loss: 0.3943 - accuracy: 0.8585
Epoch 7/30
667/667 [==============================] - 0s 537us/step - loss: 0.4190 - accuracy: 0.8510
Epoch 8/30
667/667 [==============================] - 0s 564us/step - loss: 0.4263 - accuracy: 0.8565
Epoch 9/30
667/667 [==============================] - 0s 557us/step - loss: 0.3867 - accuracy: 0.8645
Epoch 10/30
667/667 [==============================] - 0s 556us/step - loss: 0.403

In [ ]:
history.

In [12]:
model.output_shape

(None, 14)

In [36]:
model.save('../esccol_model')

INFO:tensorflow:Assets written to: ../esccol_model\assets


# SVMで学習

In [50]:
poly = PolynomialFeatures(degree=5)

In [51]:
x_poly = poly.fit_transform(x)

In [52]:
model = svm.LinearSVC(loss='hinge', C=1.0,
                      class_weight='balanced', random_state=0)

In [53]:
model.fit(x_poly,y)

c:\users\yuya\appdata\local\programs\python\python38\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(class_weight='balanced', loss='hinge', random_state=0)

In [54]:
pred_y = model.predict(x_poly)

In [55]:
correct = sum(tf.cast(pred_y==y,dtype=tf.int8))

In [56]:
correct.numpy()/len(y)

0.005